<a href="https://colab.research.google.com/github/turab45/Audd-multi-modality-for-urdu-audio/blob/master/Audd_multi_modality_for_urdu_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import cv2
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip '/content/drive/MyDrive/dataset_old.zip' -d '/content/'
#!git clone https://github.com/Nikunj1729/free-spoken-gujarati-digit-dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/7/PTT-20210602-WA1476.opus  
  inflating: /content/dataset/7/PTT-20210602-WA1478.opus  
  inflating: /content/dataset/7/PTT-20210602-WA14797.opus  
  inflating: /content/dataset/7/PTT-20210602-WA14798.opus  
  inflating: /content/dataset/7/PTT-20210602-WA14799.opus  
  inflating: /content/dataset/7/PTT-20210602-WA14800.opus  
  inflating: /content/dataset/7/PTT-20210602-WA14801.opus  
  inflating: /content/dataset/7/PTT-20210602-WA14803.opus  
  inflating: /content/dataset/7/PTT-20210602-WA14804.opus  
  inflating: /content/dataset/7/PTT-20210602-WA14805.opus  
  inflating: /content/dataset/7/PTT-20210602-WA14806.opus  
  inflating: /content/dataset/7/PTT-20210602-WA14807.opus  
  inflating: /content/dataset/7/PTT-20210602-WA1481.opus  
  inflating: /content/dataset/7/PTT-20210602-WA1483.opus  
  inflating: /content/dataset/7/PTT-20210602-WA1486.opus  
  inflating: /content/dataset/7/PTT-20210602-WA1488.op

In [94]:
# Feature extraction methods

def CalculateMelSpectrogram(file_location):
    y, sr = librosa.load(file_location)
    melSpec = librosa.feature.melspectrogram(y=y, sr=sr)
    melSpec_dB = librosa.power_to_db(melSpec)
    dim = (32, 32)
    resized = cv2.resize(melSpec_dB, dim, interpolation = cv2.INTER_AREA)
    return resized

In [13]:
cd /content

/content


In [92]:
for i in range(10):
  j = '/content/dataset'
  print(os.listdir(j))
  for k in os.listdir(j):
    print(k)
    if ".DS_" in k:
      continue

    for t in os.listdir(j+"/"+k):
      #print("path = ", os.listdir(j+"/"+k))
      if ".DS_" in k:
        continue 
      print(t.split("WA")[1].split(".")[0])
      #print(t.split("D")[1][0])
      break
    break 
  break 

['4', '3', '7', '1', '9', '2', '0', '8', '5', '6']
4
2064


In [28]:
cd free-spoken-gujarati-digit-dataset

[Errno 2] No such file or directory: 'free-spoken-gujarati-digit-dataset/S1'
/content/free-spoken-gujarati-digit-dataset


In [86]:
for i in range(5):
  j = "R" +str(i+1)+" - Central Zone"
  print("path = ",j)
  for k in os.listdir(j):
    print(k)
    
    if ".DS_" in k:
      continue 
    for t in os.listdir(j+"/"+k):
      print("path = ",j+"/"+k)
      if ".DS_" in t:
        continue
      print(t)  
      print(t.split("D"))
      print("t.split(d) -> ",t.split("D")) 
      print(t.split("D")[1][0])
      break
    break 
  break 

path =  R1 - Central Zone
S1
path =  R1 - Central Zone/S1
R1S1T2D7.wav
['R1S1T2', '7.wav']
t.split(d) ->  ['R1S1T2', '7.wav']
7


In [95]:

if not os.path.exists("processed_data"):
  os.mkdir("processed_data")
x=[]
y=[]
Total=23500
count=0
for i in range(10):
  j = '/content/dataset'
  print(os.listdir(j))
  for k in os.listdir(j):
    print(k)
    if ".DS_" in k:
      continue

    for t in os.listdir(j+"/"+k):
      #print("path = ", os.listdir(j+"/"+k))
      if ".DS_" in k:
        continue
      print(count,t)
      x.append(CalculateMelSpectrogram((j+"/"+k+"/"+t)))
      y.append(int(t.split("WA")[1].split(".")[0]))
      count+=1
    if count%300==0:
      print(Total-count)
  
x=np.array(x)
y=np.array(y)
np.save("X",x) # change these names according the feature extracted name. e.g. x_melspectrogram
np.save("y",y) # ENABLE GPU

Streaming output truncated to the last 5000 lines.
6907 AUD-20210607-WA4444.opus
6908 AUD-20210607-WA3075.opus
6909 AUD-20210607-WA2962.opus
6910 AUD-20210607-WA3014.opus
6911 AUD-20210607-WA4011.opus
6912 AUD-20210607-WA2570.opus
6913 AUD-20210531-WA1147.opus
6914 AUD-20210607-WA3657.opus
6915 AUD-20210607-WA3532.opus
6916 AUD-20210607-WA3472.opus
6917 AUD-20210607-WA3329.opus
6918 AUD-20210607-WA3783.opus
6919 AUD-20210607-WA3306.opus
6920 AUD-20210607-WA2512.opus
6921 AUD-20210607-WA4091.opus
6922 AUD-20210607-WA2517.opus
6923 AUD-20210607-WA3528.opus
6924 PTT-20210531-WA0366.opus
6925 AUD-20210607-WA2641.opus
6926 AUD-20210607-WA3417.opus
6927 AUD-20210607-WA2933.opus
6928 AUD-20210607-WA4243.opus
6929 AUD-20210607-WA2934.opus
6930 AUD-20210607-WA3780.opus
6931 AUD-20210607-WA3053.opus
6932 AUD-20210607-WA3920.opus
6933 PTT-20210601-WA1619.opus
6934 AUD-20210607-WA3097.opus
6935 AUD-20210607-WA4152.opus
6936 AUD-20210607-WA2586.opus
6937 AUD-20210607-WA4447.opus
6938 AUD-20210607-W

KeyboardInterrupt: ignored